# Batch run for zonal stats
Tips from: https://gorelick.medium.com/fast-er-downloads-a2abd512aa26

In [1]:
import os
import multiprocessing
import numpy as np
from retry import retry
import geopandas as gpd
import pandas as pd
import dask.dataframe as dd
import ee
import geemap

In [2]:
analysis_dir = '/mnt/g/Ch4/GSW_zonal_stats/v1/'
os.makedirs(analysis_dir, exist_ok=True)

In [3]:
## Register with ee using high-valume (and high-latency) endpoint
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

## Functions

In [4]:
def getRequests():
    """Generates a list of work items to be downloaded.
    Funnily enough, equivalent  to np.arange(-179.75, 179.75, 0.5)
    """    
    ## Load BAWLD
    bawld = ee.FeatureCollection('projects/ee-ekyzivat/assets/Shapes/BAWLD/bawld_grid')

    # For testing: Filter BAWLD to reduce size of operation
    # bawldF = bawld.filter("Lat > 59").filter("Lat < 60") #.filter("Long == -126.25")

    ## Aggregate by Longitude
    return np.unique(bawld.aggregate_array('Long').getInfo()) # change to bawld not bawldF for real run

In [5]:
@retry(tries=10, delay=1, backoff=2) # (tries=10, delay=1, backoff=2)
def getResult(index, long):
    """Handle the HTTP requests to download one result. index is python index and long is longitude, used for aggregation."""
    
    ## I/O
    out_dir = os.path.join(analysis_dir, 'tiles')
    out_pth = os.path.join(out_dir, f'bawld_zStats_Oc_Long{long:06.5}.csv')

    ## CRS (ist there a smarter way to do this?)
    crs = '''PROJCRS["Lambert_Azimuthal_Equal_Area",BASEGEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]],ID["EPSG",6326]],PRIMEM["Greenwich",0,ANGLEUNIT["Degree",0.0174532925199433]]],CONVERSION["unnamed",METHOD["Lambert Azimuthal Equal Area",ID["EPSG",9820]],PARAMETER["Latitude of natural origin",90,ANGLEUNIT["Degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",0,ANGLEUNIT["Degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting",south,ORDER[1],LENGTHUNIT["metre",1,ID["EPSG",9001]]],AXIS["northing",south,ORDER[2],LENGTHUNIT["metre",1,ID["EPSG",9001]]]]'''

    ## Load BAWLD
    bawld = ee.FeatureCollection('projects/ee-ekyzivat/assets/Shapes/BAWLD/bawld_grid')

    ## Load GSW
    gsw = ee.Image('JRC/GSW1_1/GlobalSurfaceWater')
    occurrence = gsw.select('occurrence')

    ## Filter based on longitude bin, specified as variable 'long'
    bawldF = bawld.filter(ee.Filter.eq("Long", long))

    # statistics_type can be either 'SUM' or 'PERCENTAGE'
    # denominator can be used to convert square meters to other areal units, such as square kilometers
    geemap.zonal_statistics_by_group(
        occurrence,
        bawldF,
        out_pth,
        statistics_type='SUM',
        denominator=1000000,
        decimal_places=3,
        crs=crs,
        tile_scale=1 # default is 1, but it exceeds GEE capacity
    )

    print("Done with Longitude: ", long)

## Apply functions

In [6]:
## View expected number of results
items = getRequests()
len(items)


NameError: name 'items' is not defined

In [ ]:
## Run function
items = getRequests()
print(f'Sending request in {len(items)} chunks...')
pool = multiprocessing.Pool(25)
pool.starmap(getResult, enumerate(items))
pool.close()
pool.join()

Sending request in 462 chunks...
Computing ... 
Computing ... 
Computing ... Computing ... 

Computing ... Computing ... 

Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Computing ... 
Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Generating URL ...
Please wait ...
Please wait ...
Generating URL ...
Please wait ...
Please wait ...
Please wait ...
Please wait ...
Please wait ...
Please wait ...
Please wait ...
Please wait ...
Please wait ...
Please wait ...
Data downloaded to /mnt/g/Ch4/GSW_zonal_stats/v0/tiles/bawld_zStats_Oc_Long-114.25.csv
Done with Longitude:  -114.25
Data downloaded to /mnt/g/Ch4/GSW_zonal_stats/v0/tiles/bawld_zStats_Oc_Long-150.75.csv
Done with Longitude:  -150.75
Computing ... 
Data downloaded to /mnt/g/Ch4/GSW_zonal_stats/v0/tiles/bawld_zStats_Oc_Long-104.25.csv
Done with Longitude:  -104.25
D

## Load and piece together

In [ ]:
# Load files using dask
# from https://mungingdata.com/pandas/read-multiple-csv-pandas-dataframe/
tile_dir = os.path.join(analysis_dir, 'tiles')
ddf = dd.read_csv(f"{tile_dir}/*.csv")

In [ ]:
## convert to pandas df
df = ddf.compute()
df

,Class_70,Class_71,Class_72,Class_73,Class_74,Class_75,Class_76,Class_77,Class_sum,Class_8,...,SGL_L,LAK,SGL,WET_L,LAR,GLA,PEB_L,Lat,RIV,system:index
0,6.702,5.305,6.165,7.046,7.096,6.910,7.825,8.163,466.767,3.338,...,1.77,13.61,2.88,2.91,0.0,0.00,1.20,64.25,0.15,00000000000000003253
1,0.228,0.100,0.328,0.301,0.276,0.167,0.334,0.268,22.280,0.233,...,0.46,4.62,0.85,11.88,0.0,0.00,2.91,68.25,0.18,00000000000000000ab3
2,2.769,3.384,3.452,3.460,2.964,4.212,3.847,3.685,394.448,6.687,...,0.81,9.95,1.44,10.79,0.0,0.00,1.10,72.25,0.18,0000000000000000122f
3,0.128,0.278,0.107,0.200,0.224,0.235,0.131,0.168,41.342,0.000,...,0.03,0.25,0.04,0.00,0.0,0.07,0.00,76.25,0.13,00000000000000000a44
4,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.04,0.87,0.34,6.12,0.0,0.00,0.98,80.25,0.15,000000000000000033ff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,0.041,0.041,0.118,0.000,0.077,0.000,0.082,0.000,22.527,0.407,...,0.00,0.00,0.00,0.50,0.0,0.00,0.00,63.25,0.21,0000000000000000076b
17,0.247,0.106,0.013,0.000,0.071,0.049,0.071,0.049,19.365,0.428,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,66.75,0.19,00000000000000001eb2
18,0.665,0.561,0.652,0.421,0.490,0.302,0.245,0.690,58.240,0.876,...,0.00,0.00,0.00,0.16,0.0,0.00,0.00,67.25,0.17,00000000000000002230
19,0.385,0.300,0.300,0.212,0.256,0.257,0.342,0.342,71.668,0.513,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,61.75,0.27,00000000000000002a50


In [ ]:
## Load shapefile to join
gdf_bawld = gpd.read_file('/mnt/g/Other/Kuhn-olefeldt-BAWLD/BAWLD/BAWLD_V1___Shapefile.zip')


In [ ]:
## Filter columns
cols_to_keep = df.columns[[('Class' in c) or ('Cell_ID' in c) for c in df.columns]]


In [ ]:

## Merge files
gdf_join = gdf_bawld.merge(df[cols_to_keep], left_on='Cell_ID', right_on='Cell_ID', validate='one_to_one')

In [ ]:
## WRite out
gdf_join_pth = os.path.join(analysis_dir, 'bawld_zStats_Oc.zip')
gdf_join.to_file(gdf_join_pth)

In [ ]:
## TODO: 
# * group by the Occurrence bins!
# * Normalize by area